# Analisi Separata per Emozione - Lessico ELIta
Questo notebook analizza le parole del lessico ELIta concentrandosi su **ciascuna delle 8 emozioni base** di Plutchik separatamente.

**Focus**:
- Solo parole (no emoji)
- Solo 8 emozioni base (no Amore, Neutrale)
- No dimensioni VAD

**Emozioni analizzate**: Gioia, Tristezza, Rabbia, Paura, Disgusto, Fiducia, Sorpresa, Aspettativa

## Setup e Caricamento Dati

In [ ]:
import pandas as pd
import emoji
import plotly.express as px
from Fase1.emotion_config import EMOTION_COLORS, BASIC_EMOTIONS
import plotly.io as pio

pio.renderers.default = "notebook_connected"

pivotedData = pd.read_csv("ELIta_INTENSITY_Matrix.csv", index_col=0)

def is_likely_emoji(text):
    return emoji.emoji_count(str(text)) > 0

pivotedData['is_emoji'] = [is_likely_emoji(x) for x in pivotedData.index]
df_words = pivotedData[pivotedData['is_emoji'] == False].copy()

df_words = df_words[BASIC_EMOTIONS]

print(f"Dataset pronto: {len(df_words)} parole testuali.")
print(f"Feature utilizzate: {BASIC_EMOTIONS}")

## Filtraggio Parole per Emozione
Decidiamo una soglia per definire l'emozione dominate di una parola.
Prima di decidere, dobbiamo guardare come sono distribuiti i dati.

In [ ]:
all_values = df_words.values.flatten() # Otteniamo tutti i valori in un array 1D
non_zero_values = all_values[all_values > 0] # Escludiamo gli 0 per la visualizzazione

fig = px.histogram(
    x=non_zero_values,
    nbins=50,
    title="Distribuzione dei punteggi di intensità (esclusi gli 0)",
    labels={'x': 'Punteggio Intensità', 'y': 'Conteggio Parole'}
)
fig.add_vline(x=0.5, line_dash="dash", line_color="red", annotation_text="Soglia 0.5")
fig.show()

print("Parole conservate con soglia > 0.5:") # Calcoliamo quante parole superano la soglia per ciascuna emozione
for emo in BASIC_EMOTIONS:
    count = len(df_words[df_words[emo] > 0.5])
    print(f"- {emo.capitalize()}: {count}")

Notiamo che la maggior parte delle parole ha punteggi bassi. Scegliamo una soglia di **0.25** (depolmente associato) per includere più parole nell'analisi.

In [ ]:
print("Parole conservate con soglia > 0.25:") # Calcoliamo quante parole superano la soglia per ciascuna emozione
for emo in BASIC_EMOTIONS:
    count = len(df_words[df_words[emo] > 0.25])
    print(f"- {emo.capitalize()}: {count}")

---
# Grafici PCA Separati per Emozione
Creiamo 8 grafici PCA distinti, uno per ogni emozione, mostrando tre soglie di intensità: 0.25, 0.5, 0.75.


In [ ]:
from IPython.display import display
from Fase1.visualizer import plot_soglie_affiancate

def analizza_emozione_completa(df, emozione, n=15):
    """
    1. Mostra il grafico PCA con soglie affiancate.
    2. Stampa la tabella delle top N parole per intensità.
    """
    print(f"--- Generazione grafici per: {emozione.upper()} ---")
    fig = plot_soglie_affiancate(df, emozione)
    fig.show()
    
    print(f"\n🔝 Top {n} parole più fortemente associate a '{emozione.capitalize()}':")
    
    top_words = df.sort_values(by=emozione, ascending=False)[[emozione]].head(n)
    
    display(top_words.style.bar(subset=[emozione], color='#d65f5f', vmin=0, vmax=1))

### Gioia

In [ ]:
analizza_emozione_completa(df_words, 'gioia')

### Tristezza

In [ ]:
analizza_emozione_completa(df_words, 'tristezza')

### Rabbia

In [ ]:
analizza_emozione_completa(df_words, 'rabbia')

### Paura

In [ ]:
analizza_emozione_completa(df_words, 'paura')

### Disgusto

In [ ]:
analizza_emozione_completa(df_words, 'disgusto')

### Fiducia

In [ ]:
analizza_emozione_completa(df_words, 'fiducia')

### Sorpresa

In [ ]:
analizza_emozione_completa(df_words, 'sorpresa')

### Aspettativa

In [ ]:
analizza_emozione_completa(df_words, 'aspettativa')

---

# Parole Ambigue
In questa sezione, identifichiamo parole che mostrano punteggi elevati in più emozioni, indicando ambiguità emotiva.

In [ ]:
from Fase1.visualizer import plot_complesse_confronto
from Fase1.pca_utils import trova_parole_complesse

fig = plot_complesse_confronto(df_words)
fig.show()

In [ ]:
complessissime = trova_parole_complesse(df_words, soglia=0.75)
complessissime

# TruncatedSVD Separato per Emozione

In [ ]:
from Fase1.visualizer import plot_svd_soglie_affiancate

fig_svd = plot_svd_soglie_affiancate(df_words, 'gioia')
fig_svd.show()

In [ ]:
fig_svd = plot_svd_soglie_affiancate(df_words, 'tristezza')
fig_svd.show()

In [ ]:
fig_svd = plot_svd_soglie_affiancate(df_words, 'rabbia')
fig_svd.show()

In [ ]:
fig_svd = plot_svd_soglie_affiancate(df_words, 'paura')
fig_svd.show()

In [ ]:
fig_svd = plot_svd_soglie_affiancate(df_words, 'disgusto')
fig_svd.show()

In [ ]:
fig_svd = plot_svd_soglie_affiancate(df_words, 'fiducia')
fig_svd.show()

In [ ]:
fig_svd = plot_svd_soglie_affiancate(df_words, 'sorpresa')
fig_svd.show()

In [ ]:
fig_svd = plot_svd_soglie_affiancate(df_words, 'aspettativa')
fig_svd.show()

---
# Matrice di Correlazione tra Emozioni
Anche se l'analisi è separata, è interessante vedere come le emozioni si correlano tra loro. 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def plot_correlation_matrix_static(df):
    
    corr = df[BASIC_EMOTIONS].corr() # Matrice di correlazione Pearson

    mask = np.triu(np.ones_like(corr, dtype=bool)) # Maschera per nascondere la metà superiore

    plt.figure(figsize=(10, 8))
    
    sns.heatmap(
        corr, 
        mask=mask, 
        annot=True,           # Scrive i numeri dentro i quadrati
        fmt=".2f",            # Formato a 2 decimali
        cmap='Reds',    
        vmin=-1, vmax=1,      # Scala da -1 a 1
        cbar_kws={"shrink": .8},
        square=True,
        linewidths=.5
    )

    plt.title("Matrice di Correlazione tra Emozioni (Pearson)", fontsize=16)
    plt.yticks(rotation=0) 
    plt.xticks(rotation=45)
    plt.show()

plot_correlation_matrix_static(df_words)